In [3]:
import requests
from datetime import *
import json
import csv
from enum import Enum
import numpy as np
import math

In [2]:
def datetime_out(dt:datetime) -> str:
	return datetime.isoformat(dt.replace(tzinfo=None),timespec='seconds') + "Z"

In [ ]:
### TIDE GAUGE

#Station 00065 (Saint John Tide gauge) has an id of 5cebf1df3d0f4a073c4bbc24
#latitude:45.254629, longitude:-66.059804
#Time series name - code (ie parameter to specify what data value we want from the api)
#Water level predictions 		- wlp
#Water level forecasts 			- wlf
#Water level official value 	- wlo
#water lavel sensor 1,2,3 		- wl1, wl2, wl3
#High and low tide predictions 	- wlp-hilo

#https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/5cebf1df3d0f4a073c4bbc24/data?time-series-code=
# wlo
# &from=2025-05-17T00%3A00%3A00Z
# &to=2025-05-17T01%3A00%3A00Z
# &resolution=FIFTEEN_MINUTES

#resolution can be one of: ALL, ONE_MINUTE, THREE_MINUTES, FIVE_MINUTES, FIFTEEN_MINUTES, SIXTY_MINUTES

# We want the official value for real time monitoring, then forecasts
# forecasts are shorter term and more accurate than predictions,
#  "include recently observed differences btw predictions and real-time observations"
# - https://tides.gc.ca/tides/en/definitions-content-tides-and-currents

tide_gauge_lat = 45.254629
tide_gauge_long = -66.059804

#ids can be found at https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations

#ids for SJ area stations: Saint John, Partridge Island, Reversing Falls, and Lorneville
# of which only SJ will have more than predictions
station_id_SJ = "5cebf1df3d0f4a073c4bbc24"
station_id_PI = "5cebf1df3d0f4a073c4bbc1d"
station_id_RF = "5cebf1de3d0f4a073c4bb9f1" # only high and low tide predictions
station_id_LV = "5cebf1e03d0f4a073c4bbe6f"

base_req = "https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/%s/data?time-series-code=%s&from=%s&to=%s&resolution=%s"


class TimeSeries(Enum):
	prediction = 1
	forecast = 2
	official = 3
	sensor1 = 4
	sensor2 = 5
	sensor3 = 7
	high_low_pred = 7

TimeSeriesText = {
	TimeSeries.prediction : "wlp",
	TimeSeries.forecast : "wlf",
	TimeSeries.official : "wlo",
	TimeSeries.sensor1 : "wl1",
	TimeSeries.sensor2 : "wl2",
	TimeSeries.sensor3 : "wl3",
	TimeSeries.high_low_pred : "wlp-hilo"
}

tide_time_res = {
	1:"ONE_MINUTE",
	3:"THREE_MINUTES",
	5:"FIVE_MINUTES",
	15:"FIFTEEN_MINUTES",
	60:"SIXTY_MINUTES"
}

def getTideData(station:str=station_id_SJ, series:TimeSeries=TimeSeries.forecast,
                start:datetime=datetime.now(), end:datetime=datetime.now()+timedelta(hours=1),
                time_res:int=15, attempts:int=3, incr:int=1):
	#Keep trying to get the data
	# if we don't get anything, try increasing the range by 'incr' minutes
	for i in range(attempts):
		req = base_req % (station, TimeSeriesText[series], start, end, tide_time_res[time_res])
		print(req)
		x = requests.get(req)
		if x.ok:
			print("Success (attempt "+str(i+1)+")")
			return json.loads(x.text)
		print("Error (attempt "+str(i+1)+"): status code was "+str(x.status_code))
	return